# Seattle Accessory Dwelling Units
Source: [City of Seattle Open Data Portal](https://data.seattle.gov/)

In [201]:
# Install Packages

# !pip install pyarrow
# !pip install geopandas
# !pip install rtree
# !pip install cenpy

# Load Packages

## General
from pprint import pprint # For tidy printing
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Storage & Filepaths
import pyarrow.feather as feather
from pathlib import Path # For storing filepaths as a Path Object

## API Pulls
# import requests
# from io import StringIO
# import time # To record elapsed time

## Geospatial
import geopandas

## Census Data 
import cenpy

## Load in ADU Data

In [202]:
data_folder = Path.cwd() / 'data' 

# ADU Data
df_aadu = pd.read_csv(data_folder /'raw'/'Detached_Accessory_Dwelling_Units_(DADUs).csv')
df_dadu = pd.read_csv(data_folder /'raw'/'Attached_Accessory_Dwelling_Units_(AADUs).csv')

# Census Data
# https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.html
vars_acs = ['B11001_001E', 'B11001_002E', 'B11017_002E', 'B25001_001E', 'B25002_003E','B25003_002E', 'B25018_001E', 'B25024_002E', 'B25024_003E', 'B25027_002E', 'B25035_001E', 'B19061_001E', 'B19013_001E', 'B19083_001E']
vars_acs_desc = ['Total HH', 'Family HH', 'Multigen HH', 'Total Units', 'Total Vac Units', 'Total Owner Occupied Units', 'Median Rooms', 'Units DADU', 'Units AADU', 'Units Mortgage', 'Median Year Built', 'Agg HH Income', 'Median HH Income', 'Gini Income Ineq']
vars_acs_dict = dict(zip(vars_acs, vars_acs_desc))

acs_seattle = (cenpy.products.ACS()
               .from_place('Seattle, WA',level='tract', variables= vars_acs)
               .drop(columns = ['state','county','tract'])
               .rename(columns = {'GEOID':'geoid20'})
               .rename(columns = vars_acs_dict)
               )
del vars_acs, vars_acs_desc

Matched: Seattle, WA to Seattle city within layer Incorporated Places


C:\Users\tbonn\AppData\Local\Temp\ipykernel_3564\2570501637.py:14: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  .from_place('Seattle, WA',level='tract', variables= vars_acs)


## Explore ADU Data

In [203]:
print(df_aadu.shape)
print(df_dadu.shape)

# Check if AADU and DADU Data have all the same varaibles
df_aadu.columns == df_dadu.columns

# Since all columns are similar -- will only examine one data set (for now)
df_aadu.head()
df_aadu.info()

(2580, 60)
(3522, 60)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 60 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OBJECTID                         2580 non-null   int64  
 1   Project Address                  2580 non-null   object 
 2   Development Site Square Feet     2574 non-null   float64
 3   Permit Number                    2580 non-null   int64  
 4   Master User Permit Number        92 non-null     float64
 5   Permit Stage                     2580 non-null   object 
 6   Type of Permit                   2580 non-null   object 
 7   New Units Permitted              2580 non-null   int64  
 8   Demolished Units Permitted       2577 non-null   float64
 9   Net Units Permitted              2577 non-null   float64
 10  Sleeping Rooms Permitted         2580 non-null   int64  
 11  Permit Value                     2556 non-null   float64
 12

## Append Data

In [204]:
# Create Source Tag Variables
df_aadu['Data Source'] = 'AADU'
df_dadu['Data Source'] = 'DADU'

# Concatenate the 2 Pandas Data Frames Together
df_adu = pd.concat([df_aadu, df_dadu], 
                   ignore_index = True, join='inner')

del df_aadu, df_dadu

## Subset & Clean Data

In [205]:
### dat_adu ###

# Subset to Variables of Interest
vars = ['Type of Dwelling Unit', 'Development Site Square Feet', 'Permit Value', 'Description of Work',
        'New Units Permitted', 'Demolished Units Permitted','Net Units Permitted', 'Sleeping Rooms Permitted',  
        'Application Date', 'Issued Date', 'Final Date', 'Most Recent Inspection Date', 
        'Project Address', 'Neighborhood', 'Council District', 'GEOID20',
        'Longitude', 'Latitude', 'Data Source']

dat_adu = df_adu[vars]

# Assess Missingness
dat_adu.isnull().sum().sort_values(ascending=False)

# Convert all Column Names to Lowercase (w/o any spaces)
dat_adu.columns = dat_adu.columns.str.lower().str.replace(' ', '_')

# Properly Format Variables

## 1) Dates: Objects --> Date Time
vars_date = ['application_date', 'issued_date', 'final_date', 'most_recent_inspection_date']
dat_adu[vars_date] = (dat_adu[vars_date]
                      .apply(lambda var: 
                             pd.to_datetime(
                                 var.str.replace(r'\d{1,2}\:\d{2}\:\d{2} [AP]M','', regex = True).str.strip() # Use regex to remove time components, strip whitespace
                                 ))) 
                                                                                                    
### Breakout Dates into Day/Month/Year Components:
for col in dat_adu.columns:
    if col.endswith('_date'):
        dat_adu[f'{col}_day'] = dat_adu[col].dt.day
        dat_adu[f'{col}_month'] = dat_adu[col].dt.month
        dat_adu[f'{col}_year'] = dat_adu[col].dt.year


## 2) Categories: Objects --> Categorical Data
vars_cat = ['type_of_dwelling_unit', 'neighborhood','council_district', 'data_source']
dat_adu[vars_cat] = dat_adu[vars_cat].apply(lambda var: var.astype('category'))

# Correct Category Naming (ADU --> AADU)
dat_adu['type_of_dwelling_unit'] = dat_adu['type_of_dwelling_unit'].cat.rename_categories({'ADU':'AADU'})

## 3) Integers: Floats --> Integers
vars_int = ['demolished_units_permitted', 'net_units_permitted']
dat_adu[vars_int] = dat_adu[vars_int].apply(lambda var: var.astype('Int64')) # Using Int64 vs int64, as its a new format which allows for NA cells within an integer variable (int64 does not)


## 4) Subset GEOID to Census Tract (15 --> 11 digits)
dat_adu['geoid20'] = dat_adu['geoid20'].astype('str').str[:-4]

del vars, vars_date, vars_cat, vars_int



### acs_seattle ###

# Convert all Column Names to Lowercase (w/o any spaces)
acs_seattle.columns = acs_seattle.columns.str.lower().str.replace(' ','_')

acs_seattle.dtypes

C:\Users\tbonn\AppData\Local\Temp\ipykernel_3564\2045449687.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat_adu[vars_date] = (dat_adu[vars_date]
C:\Users\tbonn\AppData\Local\Temp\ipykernel_3564\2045449687.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat_adu[f'{col}_day'] = dat_adu[col].dt.day
C:\Users\tbonn\AppData\Local\Temp\ipykernel_3564\2045449687.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

geoid20                         object
geometry                      geometry
total_hh                       float64
family_hh                      float64
multigen_hh                    float64
median_hh_income               float64
agg_hh_income                  float64
gini_income_ineq               float64
total_units                    float64
total_vac_units                float64
total_owner_occupied_units     float64
median_rooms                   float64
units_dadu                     float64
units_aadu                     float64
units_mortgage                 float64
median_year_built              float64
dtype: object

## Join With Census Data

In [119]:
TEST = pd.merge(dat_adu, acs_seattle, on = 'geoid20', how = 'left')

,GEOID,geometry,state,county,tract
0,53033005600,"POLYGON ((-13628336.640 6048997.580, -13627747...",53,033,005600
1,53033001500,"POLYGON ((-13626510.890 6056737.380, -13626290...",53,033,001500
2,53033001702,"POLYGON ((-13622623.050 6056364.910, -13622494...",53,033,001702
3,53033011401,"POLYGON ((-13622305.680 6028435.270, -13622305...",53,033,011401
4,53033001400,"POLYGON ((-13623770.760 6058969.720, -13623687...",53,033,001400


## Feature Engineering

In [81]:
## Calculate the time differences in the steps of the ADU building process.
dat_adu = (dat_adu
        .assign(
           processing_time = (dat_adu['issued_date'] - dat_adu['application_date']).dt.days,
            build_time = (dat_adu['final_date'] - dat_adu['issued_date']).dt.days
        ))